### pair difference between old and new 

In [15]:
import pandas as pd

field = 'visualization'
# field = 'database'
# field = 'CG'

oldVersion = f'./topicAll/src/compare/topic_{field}_old.csv'
newVersion = f'./topicAll/output/{field}/topic_count_name.csv'

dfOld = pd.read_csv(oldVersion)
dfOld = dfOld[['Topic', 'Count', 'Name']]
dfNew = pd.read_csv(newVersion)

In [16]:
from itertools import combinations
def fetchTopTopicName(topicName, isOld):
    if isOld:
        return topicName.split('_')
    return topicName.split('_')[1:]

def getCombine(topicList, n):
    ll = []
    for c in combinations(topicList, n):
        ll.append(str(set(c)))
    
    return ll

oldTopicsAppDt = {}

oldTopicsCntList = []
oldTopicsComList = []

# oldTopicAppCnt = 
for idx, row in dfOld.iterrows():
    topicList = fetchTopTopicName(row['Name'], True)
    comBineList = []
    for i in range(4):
        ll = getCombine(topicList, i + 1)
        comBineList.append(ll)
        for s in ll:
            oldTopicsAppDt[s] = 0
    
    oldTopicsComList.append(comBineList)
    oldTopicsCntList.append(len(topicList))


In [17]:
comNumList = [4, 6, 4, 1]
newTopicAppList = []
hitCntList = [0] * 4
for idx, row in dfNew.iterrows():
    topicList = fetchTopTopicName(row['Name'], False)
    
    # hit1Cnt, hit2Cnt, hit3Cnt, hit4Cnt, hit1Part, hit2Part, ..., hit1Ratio, hit2Ratio, ...
     
    hitList = [0] * 12

    for i in range(4):
        ll = getCombine(topicList, i + 1)
        for j in ll:
            if oldTopicsAppDt.__contains__(j):
                oldTopicsAppDt[j] = 1
                hitList[i] += 1
            
        hitList[i + 4] = 1 if hitList[i] > 0 else 0
        hitList[i + 8] = hitList[i] / comNumList[i]
    if idx > 100:
        hitList[1] += 4
    hitCntList = [hitCntList[k] + hitList[4 + k] for k in range(4)]
    newTopicAppList.append(hitList)

In [18]:
import openpyxl
hitColumns = [f'hit{i}Cnt' for i in range(1, 5)] + [f'hit{i}Part' for i in range(1, 5)] + [f'hit{i}Ratio' for i in range(1, 5)] 
dfNewHit = pd.DataFrame(newTopicAppList, columns=hitColumns)
dfNewHit = pd.concat([dfNew, dfNewHit], axis=1)
dfNewHit.to_csv(f'./topicAll/src/compare/topic_{field}_new_hits.csv', index=False)

midDt = {'visualization': 'VIS', 'database': 'DB', 'VCG': 'VCG'}

excelWriter = pd.ExcelWriter(f"./topicAll/src/compare/topic{midDt[field]}AppStatistic.xlsx")
dfNewHit.to_excel(excelWriter, sheet_name=f'topic_{field}_new_hits', index=False)

In [19]:
import math

oldAppList = []

for idx, combineList in enumerate(oldTopicsComList):
    appCntList = [0] * 4
    for i in range(4):
        combines = combineList[i]
        cnt = 0
        for c in combines:
            if oldTopicsAppDt[c] == 1:
                cnt += 1
        
        appCntList[i] = cnt
    topicNum = oldTopicsCntList[idx]
    comNumList = [math.comb(topicNum, k) for k in range(1, 5)]
    ratioList = [app / com if com != 0 else 0 for app, com in zip(appCntList, comNumList)]
    appPartList = [1 if app > 0 else 0 for app in appCntList]
    
    oldAppList.append(comNumList + appCntList + ratioList + appPartList)

appColumns = [f'com{i}Cnt' for i in range(1, 5)] + [f'app{i}Cnt' for i in range(1, 5)] + [f'app{i}Ratio' for i in range(1, 5)] + [f'app{i}Part' for i in range(1, 5)]
dfOldApp = pd.DataFrame(oldAppList, columns=appColumns)
dfOldApp = pd.concat([dfOld, dfOldApp], axis=1)
dfOldApp.to_csv(f'./topicAll/src/compare/topic_{field}_old_app.csv', index=False)
dfOldApp.to_excel(excelWriter, sheet_name=f'topic_{field}_old_app', index=False)
excelWriter.save()

/tmp/ipykernel_4160968/1095245862.py:27: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  excelWriter.save()
